<a href="https://colab.research.google.com/github/Observersss/numeral-methods/blob/main/lab_1/lab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import math
import numpy as np

# Параметри
EPSILON = 1e-4     # Точність
MAX_ITERATIONS = 1000  # Максимальна кількість ітерацій
INTERVAL = [-1.0, -0.5]  # Інтервал для розв'язання

# Функція та її похідні
def f(x):
    return x**3 - 10*x**2 + 44*x + 29

def f_prime(x):
    return 3*x**2 - 20*x + 44

def f_second_prime(x):
    return 6*x - 20

# Функція для методу простої ітерації
def phi(x):
    return (-x**3 + 10*x**2 - 29) / 44

def phi_prime(x):
    return (5*x)/11 - (3*x**2)/44

# Метод простої ітерації
def simple_iteration_method(x0, epsilon, max_iterations=MAX_ITERATIONS):
    x_current = x0
    for i in range(max_iterations):
        x_next = phi(x_current)
        if abs(x_next - x_current) < epsilon:
            return x_next, i + 1
        x_current = x_next
    return None, max_iterations

# Метод Ньютона
def newton_method(function, derivative_f, x0, EPSILON, max_iterations=MAX_ITERATIONS):
    x = x0
    x_vals = [x0]

    # Перевірка на відповідність умовам методу Ньютона
    if function(x0) * f_second_prime(x0) > 0:
        for i in range(max_iterations):
            if derivative_f(x) == 0:
                print("Похідна стала рівною нулю.")
                return x_vals

            x_new = x - function(x) / derivative_f(x)

            if abs(x_new - x) < EPSILON:
                x_vals.append(x_new)
                return x_vals

            x_vals.append(x_new)
            x = x_new

        print("Не вдалося досягти збіжності за максимальну кількість ітерацій.")
        return x_vals
    else:
        print("Невірна друга похідна для методу Ньютона.")
        return x_vals

# Виведення таблиці результатів для методу Ньютона
def print_table(x_vals, n_iterations, q, m1, M1, M2):
    print(f"\n{'Індекс':<10} {'Значення x':<20} {'Точність':<20} {'f(x)':<20}")
    print("=" * 80)

    for i in range(len(x_vals)):
        e = abs(x_vals[i] - x_vals[i - 1]) if i > 0 else x_vals[i]
        print(f"{i:<10} {x_vals[i]:<20.16f} {e:<20.16f} {f(x_vals[i]):<20.16f}")

    # Виведення результатів
    print(f"\nm1 (мін. абсолютна похідна): {m1}")
    print(f"M1 (макс. абсолютна похідна): {M1}")
    print(f"M2 (макс. абсолютна 2-га похідна): {M2}")
    print(f"q (фактор збіжності): {q:.4f}")
    print(f"Очікувана кількість ітерацій: {n_iterations}")

# Виведення таблиці результатів для методу простої ітерації
def print_table_simple_iteration(x_vals, n_iterations, q, delta):
    print(f"\n{'Індекс':<10} {'Значення x':<20} {'Точність':<20} {'f(x)':<20}")
    print("=" * 80)

    for i in range(len(x_vals)):
        e = abs(x_vals[i] - x_vals[i - 1]) if i > 0 else x_vals[i]
        print(f"{i:<10} {x_vals[i]:<20.16f} {e:<20.16f} {f(x_vals[i]):<20.16f}")

    # Виведення результатів
    print(f"\nq (фактор збіжності): {q:.4f}")
    print(f"delta (максимальна відстань між x0 та межами інтервалу): {delta:.4f}")
    print(f"Очікувана кількість ітерацій: {n_iterations}")


# Перевірка умов для методу Ньютона
def newton_check(x_min, x_max):
    if f(x_min) * f(x_max) < 0:
        sec_derivative_vals = np.array([f_second_prime(x) for x in np.linspace(x_min, x_max, 50)])
        if np.all(sec_derivative_vals >= 0) or np.all(sec_derivative_vals <= 0):
            return True
    return False

# Обчислення інтервалу та перевірка
def calculate_interval(interval, x0):
    x_min, x_max = interval
    if x0 < x_min or x0 > x_max:
        x0 = (x_min + x_max) / 2
        print(f"x0 поза діапазоном, змінено на {x0}")
    return x_min, x_max, x0

# Обчислення метрик для методу Ньютона
def calculate_metrics(x_min, x_max):
    x_values = np.linspace(x_min, x_max, 50)
    derivative_arr = np.abs([f_prime(x) for x in x_values])
    sec_derivative_arr = np.abs([f_second_prime(x) for x in x_values])

    m1 = np.min(derivative_arr)
    M1 = np.max(derivative_arr)
    M2 = np.max(sec_derivative_arr)

    return m1, M1, M2

# Обчислення кількості ітерацій
def calculate_iterations(delta, accuracy, q):
    primary_calc = math.log(abs(delta / accuracy)) / math.log(1 / q)
    return math.ceil(math.log2(primary_calc + 1))

# Виведення методу Ньютона
def newton_solve(interval, x0):
    x_min, x_max, x0 = calculate_interval(interval, x0)
    m1, M1, M2 = calculate_metrics(x_min, x_max)

    delta = max(abs(x_max - x0), abs(x_min - x0))

    if newton_check(x_min, x_max):
        q = (M2 * delta) / (2 * m1)
        if q < 1:
            n_iterations = calculate_iterations(delta, EPSILON, q)
            x_vals = newton_method(f, f_prime, x0, EPSILON)
            print_table(x_vals, n_iterations, q, m1, M1, M2)
        else:
            print("Збіжність не досягається, q > 1.")
    else:
        print("Перевірка умов Ньютона не пройдена.")

# Виведення методу простої ітерації
def simple_iteration_solve(interval, x0):
    x_min, x_max = interval
    x_values = np.linspace(x_min, x_max, 50)
    q = min(np.abs([phi_prime(x) for x in x_values]))

    delta = max(abs(x_max - x0), abs(x_min - x0))

    if abs(phi(x0) - x0) <= (1 - q) * delta:
        steps = math.floor((math.log(abs(phi(x0) - x0) / ((1 - q) * EPSILON)) / math.log(1 / q))) + 1
        root, steps_c = simple_iteration_method(x0, EPSILON)

        x_vals = [x0]
        for _ in range(steps_c):
            x_vals.append(phi(x_vals[-1]))
        print_table_simple_iteration(x_vals, steps, q, delta)
        print(f"Кількість ітерацій: Очікувано: {steps}, Отримано: {steps_c}")
    else:
        print("Умова збіжності для простої ітерації не виконується.")


# Виконуємо методи
print("\n Newton")
newton_solve(INTERVAL, -0.85)
print("\n Simple iteration")
simple_iteration_solve(INTERVAL, -0.6)



 Newton

Індекс     Значення x           Точність             f(x)                
0          -0.8500000000000000  -0.8500000000000000  -16.2391250000000014
1          -0.5929196184746903  0.2570803815253097   -0.8124430228530528 
2          -0.5786444570760283  0.0142751613986620   -0.0023973692278005 
3          -0.5786020837929341  0.0000423732830942   -0.0000000210717346 

m1 (мін. абсолютна похідна): 54.75
M1 (макс. абсолютна похідна): 67.0
M2 (макс. абсолютна 2-га похідна): 26.0
q (фактор збіжності): 0.0831
Очікувана кількість ітерацій: 3

 Simple iteration

Індекс     Значення x           Точність             f(x)                
0          -0.6000000000000000  -0.6000000000000000  -1.2159999999999975 
1          -0.5723636363636364  0.0276363636363636   0.3524922747287746  
2          -0.5803748244256540  0.0080111880620176   -0.1003321603213223 
3          -0.5780945480547148  0.0022802763709392   0.0287114922081102  
4          -0.5787470819685355  0.0006525339138207   -0.00